In [1]:
from langchain_community.tools import WikipediaQueryRun  ## execute a query and retrieve the results
from langchain_community.utilities import WikipediaAPIWrapper   ## set up the connection or configuration for interacting with Wikipedia.

In [2]:
api_wrapper=WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=500)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
wiki

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\AHMED ABD ELGWAD\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=500))

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load documents from the web
load_web = WebBaseLoader("https://docs.smith.langchain.com/")
docs_web = load_web.load()

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Split the documents into chunks
split_web = text_splitter.split_documents(docs_web)

# Initialize the embedding model
embedding_model = OllamaEmbeddings(model="llama3.2:3b")

# Initialize the vector database
vector_db = FAISS.from_documents(split_web, embedding_model)

# Initialize the retriever
retriever_web = vector_db.as_retriever()

retriever_web

USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\AHMED ABD ELGWAD\AppData\Local\Temp\ipykernel_11148\2330395285.py:17: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_model = OllamaEmbeddings(model="llama3.2:3b")


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024923E93DD0>, search_kwargs={})

In [ ]:
# Create a retriever tool for the vector database
from langchain.tools.retriever import create_retriever_tool         ## used to search for information in a vector database or other data source

# Create the retriever tool with the vector database and a description
retriever_web_tool = create_retriever_tool(retriever_web,
                    "langsmith_search",  ## used when integrating the tool into an agent or pipeline 
                    "Search for information about LangSmith.For any questions about LangSmith, you must use this tool!")

retriever_web_tool.name

'langsmith_search'

In [7]:
# Arxiv Tool and Utilities for Creating and Reading Applications
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun   ##  query the Arxiv database and retrieve relevant information

# Set up the connection to Arxiv API
arxiv_api = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)

# Create an ArxivQueryRun tool
arxiv_query_run = ArxivQueryRun(arxiv_api=arxiv_api)

arxiv_query_run.name

'arxiv'

In [9]:
tools = [wiki, retriever_web_tool, arxiv_query_run]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\AHMED ABD ELGWAD\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=500)),
 Tool(name='langsmith_search', description='Search for information about LangSmith.For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000249143F5620>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024923E93DD0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000024

In [ ]:
## agent
from langchain.agents import create_openai_tools_agent
